Netfix data analyzation

In [1]:
import os
from pyspark.sql.types import *
from pyspark.sql.functions import regexp_extract
from pyspark.sql.types import *
from pyspark.sql import functions as F
from pyspark.sql import functions as F
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.sql import Row
from pyspark.sql import Row

dir = 's3://dsci6007y1'
movie_file = dir + '/movie_titles.txt'
Train = dir + '/TrainingRatings.txt'
test = dir + '/TestingRatings.txt'

In [4]:
from pyspark.sql.types import *

movie_schema = StructType(  [StructField('ID', IntegerType()),   StructField('movie_year', IntegerType()),   StructField('movie_title', StringType())])
tr_schema = StructType(  [StructField('movie_Id', IntegerType()),   StructField('user_Id', IntegerType()),   StructField('movie_ratings', FloatType())])
testschema = StructType(  [StructField('movie_Id', IntegerType()),   StructField('user_Id', IntegerType()),   StructField('movie_ratings', FloatType())])


Importing libraries

In [5]:
m_t = sqlContext.read.format('csv').options(inferSchema=False).schema(movie_schema).load(movie_file)
train_r = sqlContext.read.format('csv').options(inferSchema=False).schema(tr_schema).load(Train)
test_r = sqlContext.read.format('csv').options(inferSchema=False).schema(testschema).load(test)
movie_count = m_t.count()
m_count = m_t.count()
train_count = train_r.count()
t_count = train_r.count()
Test_count = test_r.count()
tst_count = test_r.count()
print('There are ',m_count,' movietitle,',t_count,' trainingRatings and',tst_count,' testingRatings in the datasets' )

There are  17770  movietitle, 3255352  trainingRatings and 100478  testingRatings in the datasets


In [6]:
print('The Movie_titles are:')
m_t.show(4)

The Movie_titles are:
+---+----------+--------------------+
| ID|movie_year|         movie_title|
+---+----------+--------------------+
|  1|      2003|     Dinosaur Planet|
|  2|      2004|Isle of Man TT 20...|
|  3|      1997|           Character|
|  4|      1994|Paula Abdul's Get...|
+---+----------+--------------------+
only showing top 4 rows



In [7]:
print ('The TrainingRatings for the films are:')
train_r.show(4)

The TrainingRatings for the films are:
+--------+-------+-------------+
|movie_Id|user_Id|movie_ratings|
+--------+-------+-------------+
|       8|1744889|          1.0|
|       8|1395430|          2.0|
|       8|1205593|          4.0|
|       8|1488844|          4.0|
+--------+-------+-------------+
only showing top 4 rows



In [8]:
print ('The TestingRatings are:')
test_r.show(4)

The TestingRatings are:
+--------+-------+-------------+
|movie_Id|user_Id|movie_ratings|
+--------+-------+-------------+
|       8| 573364|          1.0|
|       8|2149668|          3.0|
|       8|1089184|          3.0|
|       8|2465894|          3.0|
+--------+-------+-------------+
only showing top 4 rows



In [9]:
assert movie_count == m_count
assert train_count == t_count
assert Test_count == tst_count

In [10]:
user = train_r.filter("user_Id == 2149668")
user.show()
L1 = user.select('movie_Id')
arr = [int(row.movie_Id) for row in L1.collect()]
u = train_r[train_r['movie_Id'].isin(arr)]
u.show(10)
itemsu = u.groupBy('user_Id').agg(F.count(u.movie_ratings).alias("count"),F.avg(u.movie_ratings).alias("average"))
print('movie_ids_with_avg_ratings_df:')
itemsu.show(3, truncate=False)

+--------+-------+-------------+
|movie_Id|user_Id|movie_ratings|
+--------+-------+-------------+
|     992|2149668|          3.0|
|    1202|2149668|          3.0|
|    1289|2149668|          1.0|
|    1305|2149668|          3.0|
|    2015|2149668|          5.0|
|    2212|2149668|          3.0|
|    2342|2149668|          4.0|
|    2601|2149668|          3.0|
|    2675|2149668|          3.0|
|    2755|2149668|          3.0|
|    2913|2149668|          5.0|
|    2955|2149668|          5.0|
|    3151|2149668|          4.0|
|    3253|2149668|          4.0|
|    3274|2149668|          2.0|
|    3290|2149668|          5.0|
|    3355|2149668|          5.0|
|    3538|2149668|          4.0|
|    4847|2149668|          3.0|
|    4849|2149668|          3.0|
+--------+-------+-------------+
only showing top 20 rows



+--------+-------+-------------+
|movie_Id|user_Id|movie_ratings|
+--------+-------+-------------+
|     992| 306466|          3.0|
|     992| 765331|          4.0|
|     992|  41412|          3.0|
|     992|1331887|          3.0|
|     992|1276913|          3.0|
|     992| 887273|          4.0|
|     992|1663216|          3.0|
|     992|1778851|          5.0|
|     992|2630287|          5.0|
|     992| 530441|          5.0|
+--------+-------+-------------+
only showing top 10 rows

movie_ids_with_avg_ratings_df:


[Stage 25:==============>                                           (1 + 3) / 4]22/05/03 16:13:46 WARN Utils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.debug.maxToStringFields' in SparkEnv.conf.


+-------+-----+------------------+
|user_Id|count|average           |
+-------+-----+------------------+
|128389 |42   |3.0238095238095237|
|2250628|25   |3.24              |
|279120 |24   |3.5416666666666665|
+-------+-----+------------------+
only showing top 3 rows



In [11]:
user = train_r.filter("movie_Id == 8")
user.show()
L1 = user.select('user_Id')
arr = [int(row.user_Id) for row in L1.collect()]
u = train_r[train_r['user_Id'].isin(arr)]
u.show(10)
itemsu = u.groupBy('movie_Id').agg(F.count(u.movie_ratings).alias("count"),F.avg(u.movie_ratings).alias("average"))
print('movie_ids_with_avg_ratings_df:')
itemsu.show(3, truncate=False)

+--------+-------+-------------+
|movie_Id|user_Id|movie_ratings|
+--------+-------+-------------+
|       8|1744889|          1.0|
|       8|1395430|          2.0|
|       8|1205593|          4.0|
|       8|1488844|          4.0|
|       8|1447354|          1.0|
|       8| 306466|          4.0|
|       8|1331154|          4.0|
|       8|1818178|          3.0|
|       8| 991725|          4.0|
|       8|1987434|          4.0|
|       8|1765381|          4.0|
|       8| 433803|          3.0|
|       8|1148143|          2.0|
|       8|1174811|          5.0|
|       8|1684516|          3.0|
|       8| 754781|          4.0|
|       8| 567025|          4.0|
|       8|1623132|          4.0|
|       8|1567095|          3.0|
|       8|1666394|          5.0|
+--------+-------+-------------+
only showing top 20 rows



+--------+-------+-------------+
|movie_Id|user_Id|movie_ratings|
+--------+-------+-------------+
|       8|1744889|          1.0|
|       8|1395430|          2.0|
|       8|1205593|          4.0|
|       8|1488844|          4.0|
|       8|1447354|          1.0|
|       8| 306466|          4.0|
|       8|1331154|          4.0|
|       8|1818178|          3.0|
|       8| 991725|          4.0|
|       8|1987434|          4.0|
+--------+-------+-------------+
only showing top 10 rows

movie_ids_with_avg_ratings_df:


[Stage 31:=============================>                            (2 + 2) / 4]

+--------+-----+------------------+
|movie_Id|count|average           |
+--------+-----+------------------+
|2366    |191  |2.9214659685863875|
|4190    |9    |3.4444444444444446|
|3220    |155  |2.7870967741935484|
+--------+-----+------------------+
only showing top 3 rows



In [12]:
trainingratings = train_r.toPandas()

In [13]:
rated_movie = trainingratings.groupby("user_Id")["movie_ratings"].count().sort_values(ascending = False)
rated_movie.head()

user_Id
305344     1757
387418     1744
2439493    1640
1664010    1535
2118461    1481
Name: movie_ratings, dtype: int64

In [14]:
rated_movies = trainingratings.groupby("movie_Id")["movie_ratings"].count().sort_values(ascending = False)
rated_movies.head()

movie_Id
6971    25468
6287    24393
4640    23525
9728    23184
8596    23005
Name: movie_ratings, dtype: int64

In [15]:
item_rate = trainingratings.pivot_table(index='user_Id', columns='movie_Id', values='movie_ratings')
item_rate.head()

movie_Id,8,28,43,48,61,64,66,92,96,111,...,17654,17660,17689,17693,17706,17725,17728,17734,17741,17742
user_Id,,,,,,,,,,,,,,,,,,,,,
7,5.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
79,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
199,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
481,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
769,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
item_rate.fillna(0, inplace = True)
item_rate.head()

movie_Id,8,28,43,48,61,64,66,92,96,111,...,17654,17660,17689,17693,17706,17725,17728,17734,17741,17742
user_Id,,,,,,,,,,,,,,,,,,,,,
7,5.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
79,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
199,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
481,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
769,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [17]:
Train= train_r.join(m_t,train_r.user_Id==m_t.ID)
test = test_r.join(m_t,test_r.user_Id==m_t.ID)

In [18]:
(split_60_df, split_a_20_df ) = train_r.randomSplit([7.5, 2.5])
train_df = split_60_df.cache()
validation_df = split_a_20_df.cache()
print('Training: {0}, validation: {1}\n'.format(
  train_df.count(), validation_df.count())
)
train_df.show(3)
validation_df.show(3)


Training: 2441613, validation: 813739

+--------+-------+-------------+
|movie_Id|user_Id|movie_ratings|
+--------+-------+-------------+
|       8|   1333|          3.0|
|       8|   3363|          2.0|
|       8|   3604|          3.0|
+--------+-------+-------------+
only showing top 3 rows

+--------+-------+-------------+
|movie_Id|user_Id|movie_ratings|
+--------+-------+-------------+
|       8|      7|          5.0|
|       8|   3321|          1.0|
|       8|   4706|          5.0|
+--------+-------+-------------+
only showing top 3 rows



In [19]:
als = ALS(maxIter=10, regParam=0.5, userCol="user_Id", 
                      itemCol = "movie_Id", ratingCol = "movie_ratings", coldStartStrategy = "drop")

mdl = als.fit(train_df)
prediction = mdl.transform(validation_df)
e1 = RegressionEvaluator(metricName="mse", labelCol="movie_ratings",
                                predictionCol="prediction")
                                            
e2 = RegressionEvaluator(metricName="rmse", labelCol="movie_ratings",
                                predictionCol="prediction")
rmse = e1.evaluate(prediction)
mse = e2.evaluate(prediction)
print('The mdl had a MAE on the test set of {0}'.format(mse))
print('The mdl had a RMSE on the test set of {0}'.format(rmse))

[Stage 157:================================================>    (183 + 8) / 200]

The mdl had a MAE on the test set of 1.0349200885034051
The mdl had a RMSE on the test set of 1.0710595895878958


In [20]:
pred_df = mdl.transform(test_r)
pred_test_df = pred_df.filter(pred_df.prediction != float('nan'))
test_MAE = e1.evaluate(pred_test_df)
test_RMSE = e2.evaluate(pred_test_df)
print('The mdl had a MAE on the test set of {0}'.format(test_MAE))
print('The mdl had a RMSE on the test set of {0}'.format(test_RMSE))

[Stage 217:============================================>        (169 + 8) / 200]

The mdl had a MAE on the test set of 1.0737032071438795
The mdl had a RMSE on the test set of 1.0361965099072084


In [21]:
u_rec = mdl.recommendForAllUsers(10)
m_rec = mdl.recommendForAllItems(10)
u_rec.show()
m_rec.show()

+-------+--------------------+
|user_Id|     recommendations|
+-------+--------------------+
|    481|[[3033, 4.918088]...|
|   2678|[[3033, 3.7812984...|
|   3595|[[3033, 3.3650842...|
|   6460|[[3033, 3.9558783...|
|   7284|[[3033, 4.3344574...|
|   7576|[[3033, 4.1531143...|
|   9597|[[3033, 3.5511618...|
|  15191|[[3033, 4.1148443...|
|  15846|[[3033, 4.4164057...|
|  20461|[[3033, 3.7520041...|
|  20774|[[3033, 3.5118697...|
|  26258|[[3033, 4.0454116...|
|  27608|[[3033, 4.097401]...|
|  28346|[[3033, 3.8551772...|
|  30941|[[3033, 3.7143762...|
|  30976|[[3033, 4.247151]...|
|  31203|[[3033, 3.825948]...|
|  36822|[[3033, 4.454435]...|
|  40851|[[3033, 3.921831]...|
|  41068|[[3033, 3.2386353...|
+-------+--------------------+
only showing top 20 rows



[Stage 294:==============================================>       (87 + 8) / 100]

+--------+--------------------+
|movie_Id|     recommendations|
+--------+--------------------+
|    4190|[[2307226, 3.7462...|
|    3220|[[2307226, 3.8263...|
|   11240|[[2307226, 4.1180...|
|    6110|[[2307226, 4.6255...|
|    8260|[[2307226, 3.7752...|
|     481|[[2307226, 3.9089...|
|    6911|[[2307226, 4.2951...|
|   11041|[[2307226, 4.1711...|
|   15841|[[2307226, 4.5998...|
|   15051|[[2307226, 4.4260...|
|    8851|[[2307226, 4.6340...|
|    1061|[[2307226, 4.4690...|
|   16232|[[2307226, 3.8219...|
|    9492|[[2307226, 4.3281...|
|     192|[[2307226, 4.0664...|
|    9482|[[2307226, 4.8166...|
|    6522|[[2307226, 5.3896...|
|   10082|[[2307226, 3.3696...|
|     122|[[2307226, 3.7026...|
|   16283|[[2307226, 3.2823...|
+--------+--------------------+
only showing top 20 rows



In [22]:
u_id = 1
rated_movies = [
    (u_id, 3928,  2),
    (u_id, 5326,  4),
    (u_id, 25468,  5),
    (u_id, 11088,  3),
    (u_id, 11812,  5),
    (u_id, 14185,  4),
    (u_id, 14648,  3),
    (u_id, 2290,  5),
    (u_id, 10947,  5),
    (u_id, 12293,  5)]
ratings_df = sqlContext.createDataFrame(rated_movies, ['user_Id','movie_Id','movie_ratings'])
print('My movie ratings:')
display(ratings_df.limit(8))
ratings_df.limit(8).show()

My movie ratings:


DataFrame[user_Id: bigint, movie_Id: bigint, movie_ratings: bigint]

[Stage 321:============================>                            (4 + 4) / 8]

+-------+--------+-------------+
|user_Id|movie_Id|movie_ratings|
+-------+--------+-------------+
|      1|    5326|            4|
|      1|   11088|            3|
|      1|   11812|            5|
|      1|   14648|            3|
|      1|   10947|            5|
|      1|   12293|            5|
|      1|    3928|            2|
|      1|   25468|            5|
+-------+--------+-------------+

